In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import os 
#import seaborn as sns
from itertools import zip_longest
from tqdm import tqdm
import matplotlib.lines as mlines
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import ListedColormap
import seaborn as sns
import io
from sklearn.model_selection import GroupKFold

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/Meal_distances')

In [ ]:
df_10 = pd.read_csv('meal_otsu_10_abs.csv')  
df_5 = pd.read_csv('meal_otsu_5_abs.csv')  

In [ ]:
rad = [1047, 1059, 1069, 1079, 1089, 1093, 1099, 1107, 1108, 
      1117, 1122, 1124, 1125, 1129, 1131, 1132, 1134, 1148, 1186, 
      1190, 1195, 1206, 1210, 1217, 1230, 1246, 1250, 1264, 2009, 2027]
no_rad = [1043, 1049, 1053, 1062, 1073, 1080, 1082, 1091, 1092, 
         1096, 1097, 1098, 1104, 1105, 1112, 1118, 1130, 1146, 
         1161, 1170, 1180, 1181, 1184, 1188, 1234, 1241, 1245, 1263,
         1269, 1282, 2025]

In [ ]:
def process_dataset(dataset):
    data_rad = dataset[dataset.columns & [str(x) for x in rad]]
    data_no_rad = dataset[dataset.columns & [str(x) for x in no_rad]]
    df =  pd.concat([data_rad.T.assign(label='rad'), data_no_rad.T.assign(label='no-rad')])
    df['label'] = df['label'].sort_index()
    mean, variance = df.iloc[:,:-2].mean(axis=1), df.iloc[:,:-2].var(axis=1)
    new_df = pd.concat([mean, variance], axis=1)
    new_df.columns = ['mean', 'variance']
    y = np.array([0 if x=='no-rad' else 1 for x in df['label']])
    new_df['label'] = y

    return new_df

In [ ]:
frames_5 = process_dataset(df_10)
frames_10 = process_dataset(df_5)

In [ ]:
big_df = pd.concat([frames_5, frames_10], axis=0)
label = big_df['label']

In [ ]:
big_df.index.name = 's_id'

In [ ]:
big_df = big_df.drop(columns=['label'])

In [ ]:
os.chdir('/Users/andreibirladeanu/Documents/Data/classification/speech')

In [ ]:
speech = pd.read_csv('classification.csv')  

In [ ]:
speech = speech.set_index("Subject_ID")
speech

In [ ]:
speech = speech.drop(['age', 'label', 'Proportion speech caregiver'], axis=1)
speech

In [ ]:
X = big_df.to_numpy()

y = label

In [ ]:
X_shuffled, y_shuffled, groups_shuffled = shuffle(X,y,big_df.index ,random_state=42)
print(groups_shuffled)
scalar = preprocessing.StandardScaler()
#clf = GridSearchCV(estimator=svm.SVC(), param_grid=parameter_candidates, n_jobs=-1)
parameter_candidates = {'model__kernel':['linear', 'rbf'], 'model__C':[1, 10,100]}
clf = svm.SVC()
pipeline = Pipeline(steps=[('preprocess', scalar), ('model', clf)])

cv = GroupKFold(n_splits=5)
#cross_val_score(pipeline, X, y, cv = cv)
search = GridSearchCV(pipeline, parameter_candidates, scoring = 'accuracy', cv =cv)
search.fit(X_shuffled, y_shuffled, groups = groups_shuffled)

In [ ]:
multi_modal = pd.concat([big_df, speech])
multi_modal

In [ ]:
big_df.merge(speech, on='s_id', how ='left')

In [ ]:
big_df.loc[1047]

In [ ]:
speech.loc[1047]

In [ ]:
big_df = big_df.astype('int64').dtypes


In [ ]:
speech.index


In [ ]:
big_df.index

In [ ]:
big_df.index = big_df.index.astype('int64', copy=False)

In [ ]:
big_df.index